Let's start by creating a single dataframe for all the seasons I have available

In [2]:
import pandas as pd
import numpy as np
import sys

In [3]:
#e0_1112 = pd.read_csv('../data/England/E0_1112.csv')
#e0_1213 = pd.read_csv('../data/England/E0_1213.csv')
e0_1314 = pd.read_csv('../data/England/E0_1314.csv')
e0_1415 = pd.read_csv('../data/England/E0_1415.csv')

frames = [e0_1314, e0_1415]

e0_1315 = pd.concat(frames, ignore_index=True)

e0_1315.head()
#e0_1315.tail()
#e0_1315.describe()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA
0,E0,17/08/13,Arsenal,Aston Villa,1,3,A,1,1,D,...,1.62,1.56,2.56,2.37,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,17/08/13,Liverpool,Stoke,1,0,H,1,0,H,...,1.86,1.79,2.12,2.02,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,17/08/13,Norwich,Everton,2,2,D,0,0,D,...,2.08,1.98,1.91,1.82,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,17/08/13,Sunderland,Fulham,0,1,A,0,0,D,...,2.14,2.04,1.84,1.77,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,17/08/13,Swansea,Man United,1,4,A,0,2,A,...,1.93,1.83,2.06,1.98,NaN,NaN,NaN,NaN,NaN,NaN


Create new DataFrame with the columns I'm interested in-- Team names and full-time results for now

In [114]:
df1 = e0_1315.ix[:,2:7].copy()
df1.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,Arsenal,Aston Villa,1,3,A
1,Liverpool,Stoke,1,0,H
2,Norwich,Everton,2,2,D
3,Sunderland,Fulham,0,1,A
4,Swansea,Man United,1,4,A


Create new columns to include: "lookback" Home/Away goals scored, goals conceded, and number of points

In [115]:
df1['LB_HGF'] = 0
df1['LB_AGF'] = 0
df1['LB_HGA'] = 0
df1['LB_AGA'] = 0
df1['LB_HP'] = 0
df1['LB_AP'] = 0
df1['FLAG'] = 0  #quality flag, i.e., enough matches found to compute lookback results
df1.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,LB_HGF,LB_AGF,LB_HGA,LB_AGA,LB_HP,LB_AP,FLAG
0,Arsenal,Aston Villa,1,3,A,0,0,0,0,0,0,0
1,Liverpool,Stoke,1,0,H,0,0,0,0,0,0,0
2,Norwich,Everton,2,2,D,0,0,0,0,0,0,0
3,Sunderland,Fulham,0,1,A,0,0,0,0,0,0,0
4,Swansea,Man United,1,4,A,0,0,0,0,0,0,0


Define some parameters-- for now, only the lookback_matches seems relevant

In [116]:
params = {
    'lookback': 2  #number of matches to look back at
}

#define a couple of dicts to translate results keys to points
hpoints = {'H': 3, 'D': 1, 'A': 0}
apoints = {'H': 0, 'D': 1, 'A': 3}

Suggested logic to get goals/points in X previous matches:

- iterate over the list
- get name of HomeTeam
  - start iterating back
  - look for matching name in HomeTeam
  - otherwise, in AwayTeam
  - if match is found, count goals/points
  - count a match, X-1 to go....
- get name of AwayTeam, repeat prev steps

In [117]:
for index, row in df1.iterrows():
    hometeam = row['HomeTeam']
    awayteam = row['AwayTeam']
    
    #HomeTeam
    lb_hgames = 0  #number of previous games found for home team
    lb_hgf = 0  #cumulative number of goals scored
    lb_hga = 0  #cumulative number of goals conceded
    lb_hpoints = 0  #cumulative number of points
    for j in reversed(xrange(index)):
        if(df1.loc[j,'HomeTeam']==hometeam):
            lb_hgf = lb_hgf + df1.loc[j,'FTHG']
            lb_hga = lb_hga + df1.loc[j,'FTAG']
            lb_hpoints = lb_hpoints + hpoints[df1.loc[j,'FTR']]
            lb_hgames+=1
        elif(df1.loc[j,'AwayTeam']==hometeam):
            lb_hgf = lb_hgf + df1.loc[j,'FTAG']
            lb_hga = lb_hga + df1.loc[j,'FTHG']
            lb_hpoints = lb_hpoints + apoints[df1.loc[j,'FTR']]
            lb_hgames+=1
        if(lb_hgames==params['lookback']):
            break
    df1.loc[index,'LB_HGF'] = lb_hgf
    df1.loc[index,'LB_HGA'] = lb_hga
    df1.loc[index,'LB_HP'] = lb_hpoints
    
    #AwayTeam
    lb_agames, lb_agf, lb_aga, lb_apoints = 0, 0, 0, 0  #reinitialize lookback vars
    for j in reversed(xrange(index)):
        if(df1.loc[j,'HomeTeam']==awayteam):
            lb_agf = lb_agf + df1.loc[j,'FTHG']
            lb_aga = lb_aga + df1.loc[j,'FTAG']
            lb_apoints = lb_apoints + hpoints[df1.loc[j,'FTR']]
            lb_agames+=1
        elif(df1.loc[j,'AwayTeam']==awayteam):
            lb_agf = lb_agf + df1.loc[j,'FTAG']
            lb_aga = lb_aga + df1.loc[j,'FTHG']
            lb_apoints = lb_apoints + apoints[df1.loc[j,'FTR']]
            lb_agames+=1
        if(lb_agames==params['lookback']):
            break
    df1.loc[index,'LB_AGF'] = lb_agf
    df1.loc[index,'LB_AGA'] = lb_aga
    df1.loc[index,'LB_AP'] = lb_apoints
    
    if (lb_hgames==params['lookback']) & (lb_agames==params['lookback']):
        df1.loc[index,'FLAG'] = 1

Drop rows with not enough lookback games (FLAG = 0)

In [122]:
df1 = df1[df1.FLAG != 0]
df1.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,LB_HGF,LB_AGF,LB_HGA,LB_AGA,LB_HP,LB_AP,FLAG
21,Cardiff,Everton,0,0,D,3,2,4,2,3,2,1
22,Crystal Palace,Sunderland,3,1,H,1,1,3,2,0,1,1
23,Man City,Hull,2,0,H,6,1,3,2,3,3,1
24,Newcastle,Fulham,1,0,H,0,2,4,3,1,3,1
25,Norwich,Southampton,1,0,H,2,2,3,1,1,4,1


In [118]:
df2 = df1[(df1['HomeTeam'].isin(['Arsenal'])) | (df1['AwayTeam'].isin(['Arsenal']))]
#df2.reset_index(drop=True, inplace=True)
#df2.loc[0,'FTHG']=9
df2.head()
#for index, row in df2.iterrows():
#  print index, row['HomeTeam'], row['AwayTeam']
#for i, row in enumerate(df2.values):
#    print row[1]

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,LB_HGF,LB_AGF,LB_HGA,LB_AGA,LB_HP,LB_AP,FLAG
0,Arsenal,Aston Villa,1,3,A,0,0,0,0,0,0,0
13,Fulham,Arsenal,1,3,A,1,1,0,3,3,0,0
27,Arsenal,Tottenham,1,0,H,4,2,4,0,3,6,1
36,Sunderland,Arsenal,1,3,A,2,4,4,1,1,6,1
46,Arsenal,Stoke,3,1,H,4,1,1,0,6,4,1


In [81]:
team = "Arsenal"
i = 46
df2.loc[i,:].values
lookback = 2
lb_matches = 0
lb_goals = 0

for j in reversed(xrange(i)):
    if(df1.loc[j,"HomeTeam"]==team):
        lb_goals = lb_goals + df1.loc[j,"FTHG"]
        lb_matches+=1
        #print "goals scored by arsenal: %d" %df1.loc[j,"FTHG"]
    elif(df1.loc[j,"AwayTeam"]==team):
        lb_goals = lb_goals + df1.loc[j,"FTAG"]
        lb_matches+=1
        #print "goals scored by arsenal: %d" %df1.loc[j,"FTAG"]
    if(lb_matches==lookback):
        break
    
        #print df1.loc[j,:]
print "Number of goals scored by %s in the last %d matches: %d" %(team, lookback, lb_goals)    

Number of goals scored by Arsenal in the last 2 matches: 4


In [55]:
for i in reversed(xrange(46)):
    print i

45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


In [32]:
l=[1,2,3]
for i,j in zip(l, l[1:]):
    print i, j

1 2
2 3


In [37]:
teams = df1.loc[:,'HomeTeam'].unique()
for team in teams:
    print team

Arsenal
Liverpool
Norwich
Sunderland
Swansea
West Brom
West Ham
Chelsea
Crystal Palace
Man City
Aston Villa
Everton
Fulham
Hull
Newcastle
Southampton
Stoke
Cardiff
Tottenham
Man United
Leicester
QPR
Burnley


In [58]:
df1.loc[46,"HomeTeam"]

'Arsenal'